##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 - Multimodal live API: Tool use

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/gemini-2/live_api_tool_use.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook provides examples of how to use tools with the multimodal live API with [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2).

The API provides Google Search, Code Execution and Function Calling tools. The earlier Gemini models supported versions of these tools. The biggest change with Gemini 2 (in the Live API) is that, basically, all the tools are handled by Code Execution. With that change, you can use **multiple tools** in a single API call, and the model can use multiple tools in a single code execution block.  

This tutorial assumes you are familiar with the Live API, as described in the [this tutorial](https://github.com/google-gemini/cookbook/blob/main/gemini-2/live_api_starter.ipynb).

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../gemini-2/get_started.ipynb) notebook.

In [2]:
!pip install -U -q google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 9.0 MB/s eta 0:00:00


### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

### Initialize SDK client

The client will pickup your API key from the environment variable.
To use the live API you need to set the client version to `v1alpha`.

In [4]:
from google import genai

client = genai.Client(http_options= {
      'api_version': 'v1alpha'
})

### Select a model

Multimodal Live API are a new capability introduced with the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) model. It won't work with previous generation models.

In [5]:
model_name = "gemini-2.0-flash-exp"

### Imports

In [6]:
import asyncio
import contextlib
import json
import wave

from IPython import display

from google import genai
from google.genai import types

### Utilities

You're going to use the Live API's audio output, the easiest way hear it in Colab is to write the `PCM` data out as a `WAV` file:

In [7]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

Use a logger so it's easier to switch on/off debugging messages.

In [8]:
import logging
logger = logging.getLogger('Live')
#logger.setLevel('DEBUG')  # Switch between "INFO" and "DEBUG" to toggle debug messages.
logger.setLevel('INFO')

## Get started

Most of the Live API setup will be similar to the [starter tutorial](../gemini-2/live_api_starter.ipynb). Since this tutorial doesn't focus on the realtime interactivity of the API, the code has been simplified: This code uses the Live API, but it only sends a single text prompt, and listens for a single turn of replies.

In [28]:
n = 0
async def run(prompt, modality='TEXT', tools=None):
  global n
  if tools is None:
    tools=[]

  config = {
          "tools": tools,
          "generation_config": {
              "response_modalities": [modality]}}

  async with client.aio.live.connect(model=model_name, config=config) as session:
    display.display(display.Markdown(prompt))
    display.display(display.Markdown('-------------------------------'))
    await session.send(prompt, end_of_turn=True)

    audio = False
    filename = f'audio_{n}.wav'
    with wave_file(filename) as wf:
      async for response in session.receive():
        logger.debug(str(response))
        if (text:=response.text) is not None:
          display.display(display.Markdown(text))
          continue

        if (data:=response.data) is not None:
          print('.', end='')
          wf.writeframes(pcm_data)
          audio = True
          continue

        server_content = response.server_content
        if server_content is not None:
          handle_server_content(wf, server_content)
          continue

        tool_call = response.tool_call
        if tool_call is not None:
          await handle_tool_call(session, tool_call)


  if audio:
    display.display(display.Audio(filename, autoplay=True))
    n = n+1

Since this tutorial demonstrates several tools, you'll need more code to handle the different types of objects it returns.

- The `code_execution` tool can return `executable_code` and `code_execution_result` parts.
- The `google_search` tool may attach a `grounding_metadata` object.

In [32]:
def handle_server_content(wf, server_content):
  model_turn = server_content.model_turn
  if model_turn:
    for part in model_turn.parts:
      executable_code = part.executable_code
      if executable_code is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'``` python\n{executable_code.code}\n```'))
        display.display(display.Markdown('-------------------------------'))

      code_execution_result = part.code_execution_result
      if code_execution_result is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'```\n{code_execution_result.output}\n```'))
        display.display(display.Markdown('-------------------------------'))

  grounding_metadata = getattr(server_content, 'grounding_metadata', None)
  if grounding_metadata is not None:
    display.display(
        display.HTML(grounding_metadata.search_entry_point.rendered_content))

  return

- Finally, with the `function_declarations` tool, the API may return `tool_call` objects. To keep this code minimal, the `tool_call` handler just replies to every function call with a response of `"ok"`.

In [33]:
async def handle_tool_call(session, tool_call):
  for fc in tool_call.function_calls:
    tool_response = types.LiveClientToolResponse(
        function_responses=[types.FunctionResponse(
            name=fc.name,
            id=fc.id,
            response={'result':'ok'},
        )]
    )

    print('>>> ', tool_response)
    await session.send(tool_response)

Try running it for a first time:

In [34]:
await run(prompt="Hello?", tools=None, modality = "TEXT")

Hello?

-------------------------------

Hello

 there! How can I help you today?


## Simple function call

The function calling feature of the API Can handle a wide variety of functions. Support in the SDK is still under construction. So keep this simple just send a minimal function definition: Just the function's name.

Note that in the live API function calls are independent of the chat turns. The conversation can continue while a function call is being processed.

In [35]:
turn_on_the_lights = {'name': 'turn_on_the_lights'}
turn_off_the_lights = {'name': 'turn_off_the_lights'}

In [36]:
prompt = "Turn on the lights"

tools = [
    {'function_declarations': [turn_on_the_lights, turn_off_the_lights]}
]

await run(prompt, tools=tools, modality = "TEXT")

Turn on the lights

-------------------------------

-------------------------------

``` python
print(default_api.turn_on_the_lights())

```

-------------------------------

>>>  function_responses=[FunctionResponse(id='function-call-16508437810035566102', name='turn_on_the_lights', response={'result': 'ok'})]


-------------------------------

```
{'result': 'ok'}

```

-------------------------------

OK

## Code execution

The `code_execution` lets the model write and run python code. Try it on a math problem the model can't solve from memory:

In [37]:
prompt="What is the largest prime palindrome under 100000."

tools = [
    {'code_execution': {}}
]

await run(prompt, tools=tools, modality='TEXT')

What is the largest prime palindrome under 100000.

-------------------------------

Okay

, I understand. You're asking for the largest prime number that is also

 a palindrome (reads the same forwards and backward) and is less than 1

...

Okay

, I have found the largest prime palindrome under 100,00

0.

The code generated a list of palindromes by checking every number from

 99999 downwards. It then tested each of the numbers to see if it was prime. The first prime palindrome found going downwards was 9

8689, so that is the largest one.

Therefore, the answer is 98689.


## Compositional Function Calling

Compositional function calling refers to the ability to combine user defined functions with the `code_execution` tool. The model will write them into larger blocks of code, and then pause execution while it waits for you to send back responses for each call.


In [38]:
prompt="Can you turn on the lights wait 10s and then turn them off?"

tools = [
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights, turn_off_the_lights]}
]

await run(prompt, tools=tools, modality='TEXT')

Can you turn on the lights wait 10s and then turn them off?

-------------------------------

-------------------------------

``` python
import time
default_api.turn_on_the_lights()
time.sleep(10)
default_api.turn_off_the_lights()

```

-------------------------------

>>>  function_responses=[FunctionResponse(id='function-call-6729032840971697927', name='turn_on_the_lights', response={'result': 'ok'})]
>>>  function_responses=[FunctionResponse(id='function-call-15373898631942882332', name='turn_off_the_lights', response={'result': 'ok'})]


## Google search

The `google_search` tool lets the model conduct google searches. For example, try asking it about events that are too recent to be in the training data.

The search still executes in `AUDIO` mode, but you don't see the detailed results. So switch to text mode to see the full output:

In [40]:
prompt="Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?"

tools = [
   {'google_search': {}}
]

await run(prompt, tools=tools, modality='TEXT')

Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?

-------------------------------

-------------------------------

``` python
print(google_search.search(queries=["largest earthquake in California week of December 5 2024", "California earthquakes week of December 5 2024"]))

```

-------------------------------

...

   **Date:** December 5, 2024
*   **Time:** 10:44 a.m. PT
*   **Location:** Offshore of Cape Mendocino, about 60 miles

 southwest of Ferndale, Northern California
*   **Tectonic Setting:** The earthquake occurred west of the Mendocino triple junction, where the Pacific, North America, and Juan de Fuca/Gorda plates meet, along the Mend

ocino fracture zone.
*   **Tsunami Warning:** The earthquake triggered a brief tsunami warning for coastal areas from Santa Cruz into Oregon, which was later canceled. The warning impacted more than 4.6 million people.
*   **Impact:** While the earthquake was strong, it did not cause major damage

. Some areas reported a rolling motion for several seconds. Weak to light shaking was reported in various locations, including the Bay Area, Santa Cruz, Eureka, and Sacramento.

It is worth noting that the area around the Mendocino triple junction is one of the more seismically active regions in California, with several magnitude

 7 or greater earthquakes occurring there since 1900.


## Multi-tool


The biggest difference with the new API however is that you're no longer limited to using 1-tool per request. Try combining those tasks from the previous sections:

In [41]:
prompt = """\
  Hey, I need you to do three things for me.

  1. Then compute the largest prime plaindrome under 100000.
  2. Then use google search to lookup unformation about the largest earthquake in california the week of Dec 5 2024?
  3. Turn on the lights

  Thanks!
  """

tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights, turn_off_the_lights]}
]

await run(prompt, tools=tools, modality="TEXT")

  Hey, I need you to do three things for me.

  1. Then compute the largest prime plaindrome under 100000.
  2. Then use google search to lookup unformation about the largest earthquake in california the week of Dec 5 2024?
  3. Turn on the lights

  Thanks!
  

-------------------------------

Okay

, I will perform those tasks for you. First, let's find the

 largest prime palindrome under 100000.


...

. It appears to be the largest earthquake in that region for the week in question.

Finally, I will turn on the lights.


-------------------------------

``` python
default_api.turn_on_the_lights()

```

-------------------------------

>>>  function_responses=[FunctionResponse(id='function-call-5847648409604916552', name='turn_on_the_lights', response={'result': 'ok'})]


## Next Steps

- For more information about the SDK see the [SDK docs](https://googleapis.github.io/python-genai/)
- This tutorial uses the high level SDK, if you're interested in the lower-level details, try the [Websocket version of this tutorial](../gemini-2/websocket/search_tool.ipynb)
- This tutorial only covers _basic_ usage of these tools for deeper (and more fun) example see the [Search tool tutorial](../gemini-2/search_tool.ipynb)

Or check the other Gemini 2.0 capabilities from the [Cookbook](https://github.com/google-gemini/cookbook/blob/main/gemini-2/), in particular this other [multi-tool](../gemini-2/plotting_and_mapping.ipynb) example and the one about Gemini [spatial capabilities](../gemini-2/spatial_understanding.ipynb).